## INTRODUCTION TO CNN USING TENSORFLOW

### BASIC EXAMPLE

![BASIC CNN](./images/cnn-basic-image)

### STEP 1: IMPORT LIBRARY AND LOAD DATASET

In [1]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


### STEP 2: CREATE AN INTERACTIVE SESSION

In [2]:
sess = tf.InteractiveSession()

### STEP 3: DEFINE THE SHAPE OF OUR DATASET

In [3]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

### STEP 4: DEFINE HELPER FUNCTIONS

#### 1. FOR WEIGHTS AND BIASES

In [4]:
W = lambda shape: tf.Variable(tf.truncated_normal(shape, stddev=0.1))
b = lambda shape: tf.Variable(tf.constant(0.1, shape=shape))

#### 2. FOR CONVOLUTION AND POOLING LAYERS

In [5]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

### STEP 5: DEFINE MODEL

#### OPTIMISATION 1:

- Uptil now we have been ignoring one pattern which has been in our dataset... That is the spatial data...
- That is if instead of considering the input data points as independant data units... let's us consider that these points are dependant on their neighbours and think of the data as a 2d array of pixels.
- CNNs exploit this spatial factor to improve the accuracy of any model provided the data points have inherent spatial data.

#### LAYER 1: CONVOLUTIONAL LAYER WITH MAX POOLING

In [6]:
W_conv1 = W(shape=[5, 5, 1, 32])
b_conv1 = b(shape=[32])
x_conv1 = tf.reshape(x, [-1, 28, 28, 1])

conv1 = tf.nn.relu(conv2d(x_conv1, W_conv1) + b_conv1)
pool1 = max_pool_2x2(conv1)

#### LAYER 2: CONVOLUTIONAL LAYER WITH MAX POOLING

In [7]:
W_conv2 = W(shape=[5, 5, 32, 64])
b_conv2 = b(shape=[64])

conv2 = tf.nn.relu(conv2d(pool1, W_conv2) + b_conv2)
pool2 = max_pool_2x2(conv2)

#### LAYER 3: DENSELY CONNECTED LAYER

In [8]:
W_fc1 = W(shape=[7 * 7 * 64, 1024])
b_fc1 = b(shape=[1024])

pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
fc1 = tf.nn.relu(tf.matmul(pool2_flat, W_fc1) + b_fc1)

### DROPOUT

In [9]:
keep_prob = tf.placeholder(tf.float32)
fc1_drop = tf.nn.dropout(fc1, keep_prob)

#### LAYER 4: READOUT LAYER

In [10]:
W_fc2 = W(shape=[1024, 10])
b_fc2 = b(shape=[10])

y_conv = tf.matmul(fc1, W_fc2) + b_fc2

### STEP 6: DEFINE METRICS

#### 1. LOSS IN THE FORM OF CROSS ENTROPY

In [11]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))

#### 2. TRAINING USING ADAM OPTIMIZER

In [12]:
global_step = tf.Variable(0, trainable=False)
learning_rate = tf.train.exponential_decay(
    learning_rate=0.01,
    global_step=global_step,
    decay_steps=10000,
    decay_rate=0.50
)

In [13]:
train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy, global_step)

#### 3. ACCURACY OF THE MODEL

In [14]:
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

### STEP 7: TRAIN MODEL

In [15]:
sess.run(tf.global_variables_initializer())

In [16]:
for i in range(1000):
    batch = mnist.train.next_batch(50) 
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
    
    ### TEST OUR MODEL INBETWEEN
    if i % 100 == 0:
        train_accuracy = accuracy.eval(feed_dict={ x: batch[0], y_: batch[1], keep_prob: 1 })
        print('STEP', i, ':', 'ACCURACY', train_accuracy)

STEP 0 : ACCURACY 0.22
STEP 100 : ACCURACY 0.98
STEP 200 : ACCURACY 0.98
STEP 300 : ACCURACY 0.98
STEP 400 : ACCURACY 1.0
STEP 500 : ACCURACY 0.94
STEP 600 : ACCURACY 0.98
STEP 700 : ACCURACY 0.96
STEP 800 : ACCURACY 0.98
STEP 900 : ACCURACY 1.0


### STEP 8: EVALUATE MODEL

In [17]:
final_accuracy = accuracy.eval(feed_dict={ x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1 })
print('FINAL ACCURACY:', final_accuracy)

FINAL ACCURACY: 0.9751


### STEP 9: TEST PREDICTION

In [18]:
x_actual = mnist.test.images[0:1]
y_actual = mnist.test.labels[0:1]

print('ACTUAL VALUE:', y_actual)

ACTUAL VALUE: [[ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]]


In [19]:
y_predicted = sess.run(y_conv, feed_dict={x: x_actual, keep_prob: 1})

print('PREDICTED VALUE:', y_predicted)

PREDICTED VALUE: [[  2.41497111   0.27594382   6.96642971   5.32868099   2.03751612
    2.52723384  -4.13650703  16.62407684   3.11290574   5.37490654]]


In [20]:
output_actual = sess.run(tf.argmax(y_actual, 1))
output_predicted = sess.run(tf.argmax(y_predicted, 1))

print ('EXPECTED OUTPUT:', output_actual)
print ('PREDICTED OUTPUT:', output_predicted)

EXPECTED OUTPUT: [7]
PREDICTED OUTPUT: [7]
